# Artificial Intelligence
# 464/664
# Assignment #6

## General Directions for this Assignment

00. We're using a Jupyter Notebook environment (tutorial available here: https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html),
01. Read the entire notebook before beginning your work, 
02. Output format should be exactly as requested (it is your responsibility to make sure notebook looks as expected on Gradescope),
03. Each helper function should be preceeded by documentation (Markdown cell), 
04. Each helper function (not `train`) should be followed by three assert-style unit tests,
05. **Do not use any AI/ML libraries, packages, such as pandas, scikit (numpy is fine)**
06. Functions should do only one thing,
07. Check submission deadline on Gradescope, 
08. Rename the file to Last_First_assignment_6, 
09. Submit your notebook (as .ipynb, not PDF) using Gradescope, and
10. Do not submit any other files.

## Before You Submit...

1. Re-read the general instructions provided above, and
2. Hit "Kernel"->"Restart & Run All".

## Decision Trees

For this assignment we will implement a Decision Tree using the ID3 Algorithm. The goal is classify a mushroom as either edible ('e') or poisonous ('p'). Dataset has been uploaded to Canvas. In case you'd like to learn more about it, here's the link to the repo: https://archive.ics.uci.edu/dataset/73/mushroom. 


Our  Decision Tree pipeline is as follows:


1) `cross_validate` will take data (supplied as folds using 10 fold cross validation) and do the following:
* For each setting of depth limit (the hyperparameter in decision trees, including 0)
* * and for each fold of data
* * * use `create_train_test` to split current fold into train and test
* * * call `train` to build and return a decision tree, 
* * * call `classify` to use the tree to get classifications,
* * * call `evaluate` to compare classifications to the actual answers (ground truth),
* * * Print the performance for that fold
* * Summarize the performance for that depth limit over all folds using `get_stats`


2) `pretty_print_tree(tree)` will print what the tree looks like when using the **entire** data set (no train/test split) with depth limit set to None.


All the code in this pipeline has been provided, except for a working `train` function. The `train` function currently returns a hard-coded tree from our lecture. Don't do that. Use ID3 to build your tree and use the depth limit to stop. When you're train function is complete, it should work for the lecture data, and mushrooms. Although `train` is terrible right now, pay attention to how the tree is structured.

In [ ]:
import random
import math
import copy
from copy import deepcopy
from typing import List, Dict, Tuple, Callable

import numpy

<a id="note"></a>

<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Note</strong>
    <p>
        Let's start with our example from the 06-Nov lecture. Target variable is Safe?, which can be yes or no. Anything *_lecture refers to the dataset we walked through in class.  
    </p>
</div>

In [ ]:
data_lecture = [['round','large','blue','no'],
['square','large','green','yes'],
['square','small','red','no'],
['round','large','red','yes'],
['square','small','blue','no'],
['round','small','blue','no'],
['round','small','red','yes'],
['square','small','green','no'],
['round','large','green','yes'],
['square','large','green','yes'],
['square','large','red','no'],
['square','large','green','yes'],
['round','large','red','yes'],
['square','small','red','no'],
['round','small','green','no']]

In [ ]:
print(data_lecture[0]) # a record of data

In [ ]:
len(data_lecture)

In [ ]:
attribute_names_lecture = ['shape', 
                      'size', 
                      'color']

<a id="create_folds"></a>
## create_folds


With n-fold cross validation, we divide our data set into n subgroups called "folds" and then use those folds for training and testing. For data set with 100 observations (or records), n set to 10 would have 10 observations in each fold.

* **data** List: a list (data_lecture, for instance)
* **n** int: number of folds


**returns** 
folds, which is a list of n items, where each item is a list containing a subgroup of xs

In [ ]:
def create_folds(data: List, n: int) -> List[List[List]]:
    k, m = divmod(len(data), n)
    return list(data[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))

In [ ]:
folds_lecture = create_folds(data=data_lecture, n=10)

In [ ]:
len(folds_lecture)

In [ ]:
print(folds_lecture[0])

In [ ]:
print(folds_lecture[1])

<a id="create_train_test"></a>
## create_train_test


This function takes the n folds and returns the train and test sets. One of the n folds is used to test, the others are used for training.

* **folds** List[List[List]]: see `create_folds`
* **index** int: fold index that is used for testing


**returns** 
folds, which is a list of n items, where each item is a list containing a subgroup of xs

In [ ]:
def create_train_test(folds: List[List[List]], index: int) -> Tuple[List[List], List[List]]:
    training = []
    test = []
    for i, fold in enumerate(folds):
        if i == index:
            test = fold
        else:
            training = training + fold
    return training, test

In [ ]:
train_lecture, test_lecture = create_train_test(folds_lecture, 0) # test data is folds_lecture index 0

In [ ]:
print(train_lecture)

In [ ]:
print(test_lecture)

In [ ]:
train_lecture, test_lecture = create_train_test(folds_lecture, 1) # test data is folds_lecture index 1

In [ ]:
print(train_lecture)

In [ ]:
print(test_lecture)

<a id="note"></a>

<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <p>
        Let's load the mushroom data.
    </p>
</div>

<a id="parse_data"></a>
## parse_data

Opens a file, splits on comma, and shuffles data before returning as a List of list. 

* **file_name** Str: filename for data


**returns** 
Data as a list of a list.

In [ ]:
def parse_data(file_name: str) -> List[List]:
    data = []
    file = open(file_name, "r")
    for line in file:
        datum = [value for value in line.rstrip().split(",")]
        data.append(datum)
    random.shuffle(data)
    return data

In [ ]:
data_mushroom = parse_data("agaricus-lepiota.data")

<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
        We're going to move the target column (mushroom edible or poisonous) to the last column to match the lecture's format, where Safe? was at the end.
    </p>
</div>

In [ ]:
data_mushroom = [record[1:]+[record[0]] for record in data_mushroom]

In [ ]:
len(data_mushroom)

In [ ]:
print(data_mushroom[0])

In [ ]:
attribute_names_mushroom = ['cap-shape',
                   'cap-surface',
                   'cap-color',
                   'bruises?',
                   'odor',
                   'gill-attachment',
                   'gill-spacing',
                   'gill-size',
                   'gill-color',
                   'stalk-shape',
                   'stalk-root',
                   'stalk-surface-above-ring',
                   'stalk-surface-below-ring',
                   'stalk-color-above-ring',
                   'stalk-color-below-ring',
                   'veil-type',
                   'veil-color',
                   'ring-number',
                   'ring-type',
                   'spore-print-color',
                   'population',
                   'habitat']

<a id="get_answers"></a>
## get_answers

This function extracts a list of the target values from data. The function assumes the target variable is the last column of the data.

* **data** List[List]: The data provided in a list of list format identical to the structure of `data_lecture` or `data_mushroom`


**returns** 
A list of the values of the target variable.

In [ ]:
def get_answers(data):
    return [record[-1] for record in data]

In [ ]:
assert get_answers([]) == []
assert get_answers(data_lecture) == ['no', 'yes', 'no', 'yes', 'no', 'no', 'yes', 'no', 'yes', 'yes', 'no', 'yes', 'yes', 'no', 'no']

<a id="get_mode"></a>
## get_mode

This function finds the mode of a list of items. 

* **answers** List: A list of items

**returns** 
The item that appears the most often in the list. 

In [ ]:
def get_mode(answers):
    count_dict = {}
    for answer in answers:
        if answer in count_dict:
            count_dict[answer] = count_dict[answer] + 1
        else:
            count_dict[answer] = 1
    mode_count = max(count_dict.values())
    mode = [k for k, v in count_dict.items() if v == mode_count]
    return mode[0]

In [ ]:
assert get_mode(['no', 'no', 'no', 'yes']) == 'no'
assert get_mode(['no', 'no', 'yes', 'yes']) == 'no'
assert get_mode(['no', 'yes', 'yes', 'yes']) == 'yes'

## retrieve_feature_choices_and_index_map
Function that finds the possible choices for a given feature in context of dataset, along with a mapping between feature name and associated index based on attribute list.

* **data** List[List[str]]: list of records that contain value for each feature in attribute names along with associated target value.
* **attribute_names** List[str]: list of features, i.e attributes, defined for given dataset.


* **RETURN** Tuple[Dict[str, Set[str]]], Dict[str, int]: yields a map from feature to possible choices, along with anohter map from feature to dataset column index.

In [ ]:

def retrieve_feature_choices_and_index_map(data, attribute_names):
  feature_choices = {}
  feature_index_map = {}

  for i, feature in enumerate(attribute_names):
     feature_choices[feature] = set()
     feature_index_map[feature] = i
     
  for col in range(len(data[0]) - 1):
      feature = attribute_names[col]
      for row in range(len(data)):
         val = data[row][col]
         feature_choices[feature].add(val)

  return feature_choices, feature_index_map

In [ ]:
test_feat_choices_1, test_feature_index_map_1 = retrieve_feature_choices_and_index_map(data_lecture, attribute_names_lecture)

assert(test_feat_choices_1 == {"size": {"large", "small"}, "color": {"blue", "red", "green"}, "shape": {"round", "square"}})
assert(test_feature_index_map_1 == {"shape": 0, "size": 1, "color": 2})


test_data_cats_humans = [
                ['yes', 'no', 'no', 'human'],
                ['no', 'yes', 'yes', 'cat'],
                ['no', 'yes', 'no', 'human']

                ]
test_attr_names_cats_humans = ['intelligent?', 'lazy?', 'fur?']

test_feat_choices_2, test_feature_index_map_2 = retrieve_feature_choices_and_index_map(test_data_cats_humans, test_attr_names_cats_humans)
assert(test_feat_choices_2 == {"intelligent?": {"yes", "no"}, "lazy?": {"yes", "no"}, "fur?": {"yes", "no"}})
assert(test_feature_index_map_2 == {"intelligent?": 0, "lazy?": 1, "fur?": 2})


test_data_job_candidates = [
                ['<=1', 'bachelors', 'no', 'decline'],
                ['<=5', 'phd', 'yes', 'hire'],
                ['<=10', 'masters', 'no', 'hire'],
                ['<=1', 'masters', 'yes', 'hire']
                ]
test_attr_names_job_candidates = ['yearsExperience', 'education', 'requireSponsorship?']
test_feat_choices_3, test_feature_index_map_3 = retrieve_feature_choices_and_index_map(test_data_job_candidates, test_attr_names_job_candidates)
assert(test_feat_choices_3 == {"yearsExperience": {"<=1", "<=5", "<=10"}, "education": {"bachelors", "masters", "phd"}, "requireSponsorship?": {"yes", "no"}})
assert(test_feature_index_map_3 == {"yearsExperience": 0, "education": 1, "requireSponsorship?": 2})

## retrieve_target_sizes

Function that calculates the number of occurences for all possible target values that exist in dataset.

* **data** List[List[str]]: list of records that contain value for each feature in attribute names along with associated target value.
* **target_types** List[str]: list of all the possible target variable values.


* **RETURN** Tuple[int, int]: yields the number of occurences for both target value types hat exist in dataset.

In [ ]:
def retrieve_target_sizes(data, target_types): # Only accounts for supervised learning classification with TWO types
  target_sz_1, target_sz_2 = 0, 0
  for record in data:
        target_val = record[-1]
        if target_val == target_types[0]: 
          target_sz_1 += 1
        elif target_val == target_types[1]:
          target_sz_2 += 1
  return target_sz_1, target_sz_2

In [ ]:
test_1_target_sz_1, test_1_target_sz_2 = retrieve_target_sizes(data_lecture, ["yes", "no"])
assert(test_1_target_sz_1 == 7 and test_1_target_sz_2 ==8)

test_data_cats_humans = [
                ['yes', 'no', 'no', 'human'],
                ['no', 'yes', 'yes', 'cat'],
                ['no', 'yes', 'no', 'human']

                ]
test_targets_cats_humans = ['human', 'cat']

test_2_target_sz_1, test_2_target_sz_2 = retrieve_target_sizes(test_data_cats_humans, test_targets_cats_humans)
assert(test_2_target_sz_1 == 2 and test_2_target_sz_2 == 1)

test_data_job_candidates = [
                ['<=1', 'bachelors', 'no', 'decline'],
                ['<=5', 'phd', 'yes', 'hire'],
                ['<=10', 'masters', 'no', 'hire'],
                ['<=1', 'masters', 'yes', 'hire']
                ]
test_targets_job_candidates = ['hire', 'decline']

test_3_target_sz_1, test_3_target_sz_2 = retrieve_target_sizes(test_data_job_candidates, test_targets_job_candidates)
assert(test_3_target_sz_1 == 3 and test_3_target_sz_2 == 1)


## entropy_hx
Function that calculates the suprise in a dataset based on entropy equation H(x), utilizing the the number of target types in proportion to the size of dataset.
* **target_sz_1** int: the number of occurences for one of target values in a dataset.
* **target_sz_2** int: the number of occurences for the other target value in a dataset.
* **proportion_denom** int:  size of dataset, typically denoted size as dataset that contains a specified choice for a feature.


* **RETURN** float: returns the calculated entropy or suprise in dataset.

In [ ]:
def entropy_hx(target_sz_1, target_sz_2, proportion_denom): # Only accounts for supervised learning classification with TWO types
  if target_sz_1 + target_sz_2 != proportion_denom:
    print("Error: proportions sum don't equare to 1")
    return None
  
  if target_sz_1 == 0 or target_sz_2 == 0: return 0.0
  
  p_1, p_2 = float(target_sz_1 / proportion_denom), float(target_sz_2 / proportion_denom)

  target_entropy_1 = -p_1 * float(numpy.log2(p_1)) * p_1 
  target_entropy_2 = -p_2 * float(numpy.log2(p_2)) * p_2

  return target_entropy_1 + target_entropy_2 

In [ ]:
test_proportion_denom = 8 # Test different proportions against a common denominator

test_1_target_sz_1 = 4
test_1_target_sz_2 = 4

test_entropy_1 = entropy_hx(test_1_target_sz_1, test_1_target_sz_2, test_proportion_denom)
assert(test_entropy_1 == 0.5)


test_2_target_sz_1 = 8
test_2_target_sz_2 = 0

test_entropy_2 = entropy_hx(test_2_target_sz_1, test_2_target_sz_2, test_proportion_denom)
assert(test_entropy_2 == 0.0)

test_3_target_sz_1 = 2
test_3_target_sz_2 = 6

test_entropy_3 = entropy_hx(test_3_target_sz_1, test_3_target_sz_2, test_proportion_denom)
assert(test_entropy_3 == 0.3584585933443496)

## retrieve_target_types
Function that retrieves all the different types of target values.

* **data** List[List[str]]: list of records that contain value for each feature in attribute names along with associated target value.

* **RETURN** Tuple[List[str], Set[str]]: all the unique types.

In [ ]:
def retrieve_target_types(data): # Only accounts for supervised learning classification with TWO types
  answers = get_answers(data)
  types = set() # Use set to disregard duplicated target values in data

  for target_val in answers:
    if len(types) == 2: break
    types.add(target_val)

  return types

In [ ]:
test_types_1 = retrieve_target_types(data_lecture)
assert(test_types_1 == {"yes", "no"})

test_types_2 = retrieve_target_types(data_mushroom)
assert(test_types_2 == {"p", "e"})

test_data_job_candidates = [
                ['<=1', 'bachelors', 'no', 'decline'],
                ['<=5', 'phd', 'yes', 'hire'],
                ['<=10', 'masters', 'no', 'hire'],
                ['<=1', 'masters', 'yes', 'hire']
                ]

test_types_3 = retrieve_target_types(test_data_job_candidates)
assert(test_types_3 == {"hire", "decline"})

## retrieve_feature_choice_subset
Function that generates a subset for a dataset for all records that contain the choice or rather the specified value for chosen feature.

* **data** List[List[str]]: list of records that contain value for each feature in attribute names along with associated target value.
* **choice** str: the specified value for a given feature.
* **feature_col** int: column index marking the feature we're looking at in the dataset.


* **RETURN** List[List[str]]: the of records that contain specified value for a chosen feature.

In [ ]:
def retrieve_feature_choice_subset(data, choice, feature_col):
  subset = []
  for record in data:
    if record[feature_col] == choice:
      subset.append(record)
  
  return subset

In [ ]:
test_feature_choice_sub_1 = retrieve_feature_choice_subset(data_lecture, "round", 0)
assert(test_feature_choice_sub_1 == [
                                      ['round','large','blue','no'],
                                      ['round','large','red','yes'],
                                      ['round','small','blue','no'],
                                      ['round','small','red','yes'],
                                      ['round','large','green','yes'],
                                      ['round','large','red','yes'],
                                      ['round','small','green','no']
                                    ]
      )


test_feature_choice_sub_2 = retrieve_feature_choice_subset(test_feature_choice_sub_1, "large", 1)

assert(test_feature_choice_sub_2 == [
                                      ['round','large','blue','no'],
                                      ['round','large','red','yes'],
                                      ['round','large','green','yes'],
                                      ['round','large','red','yes'],
                                    ]
      )


test_feature_choice_sub_3 = retrieve_feature_choice_subset(test_feature_choice_sub_2, "red", 2)
assert(test_feature_choice_sub_3 == [
                                      ['round','large','red','yes'],
                                      ['round','large','red','yes'],
                                    ]
      )

## retrieve_entropy_weighted_avg
Function that calculates the weighted average for the feature entropy considering a given feature in the id3 run.

* **choice_entropy_map** Dict[str, Tuple(float, int)]: dictionary that maps a choice and it's calculated entropy in a subset along with the subset size with containing that choice.
* **baseline_sz** int: the size of the dataset before considering any choice for a feature we're considering at moment of id3 run.


* **RETURN** float: the weighted average among all the choice entropies when considering a given feature in the id3 run.

In [ ]:
def retrieve_entropy_weighted_avg(choice_entropy_map, baseline_sz):
  entropy_weight_avg = 0.0

  for choice in choice_entropy_map:
    entropy, choice_sz = choice_entropy_map[choice]
    weight = float(choice_sz / baseline_sz) 
    entropy_weight_avg += float(weight * entropy)
    
  return entropy_weight_avg

In [ ]:
test_baseline_sz = 8
test_choice_entropy_map_1 = {"small": (1.0, 8), "large": (0.0, 8)}   

test_weighted_entropy_1 = retrieve_entropy_weighted_avg(test_choice_entropy_map_1, test_baseline_sz)
assert(test_weighted_entropy_1 == 1.0)


test_choice_entropy_map_2 = {"small": (0.5, 4), "large": (0.5, 4)}   
test_weighted_entropy_2 = retrieve_entropy_weighted_avg(test_choice_entropy_map_2, test_baseline_sz)

assert(test_weighted_entropy_2 == 0.5)

test_choice_entropy_map_2 = {"small": (0.5, 4), "large": (0.5, 2)}   
test_weighted_entropy_2 = retrieve_entropy_weighted_avg(test_choice_entropy_map_2, test_baseline_sz)

assert(test_weighted_entropy_2 == 0.375)


## retrive_best_feature
* **subset** List[List[str]]: a list of relevant records constrained to  prior selected features and corresponding choices.
* **feature_choices** Dict[str, Set[str]]: a map with pairs of features pointing to its possible choices.
* **feature_index_map** Dict[str, int]: a map with pairs of features pointing to it's correseponding dataset column index.
* **remaining_feats** Set[str]: collection of remaining attributes to find best feature from.
* **target_types** List[str]: collection of the unique types of target variables in subset.
* **baseline_sz** int: the size of the subset evaluated from prior feature choice(s) before examining remaining features.
* **baseline_entropy** float: the entropy of the subset constrained to feature choice(s) chosen before examining remaining features.

* **RETURN** Tuple[str, Dict[str, Tuple[float, float]]]: the next best feature.

In [ ]:
def retrive_best_feature(subset, feature_choices, feature_index_map, remaining_feats, target_types, baseline_sz, baseline_entropy):
  max_info_gain, best_feat = float("-inf"), None

  for feature in remaining_feats: 
    choice_entropy_map = {}

    for choice in feature_choices[feature]:
      # Get partition of subset that contains current feature's choice
      subset_with_feat_choice = retrieve_feature_choice_subset(subset, choice, feature_col=feature_index_map[feature]) 
      choice_sz = len(subset_with_feat_choice)

      # Count up the size for both types of target values per choice made for a feature
      target_sz_1, target_sz_2 = retrieve_target_sizes(subset_with_feat_choice, target_types)
      
      # Calculate entropy per choice made for a feature
      choice_entropy = entropy_hx(target_sz_1, target_sz_2, choice_sz)
      choice_entropy_map[choice] = (choice_entropy, choice_sz)
    
    feature_entropy = retrieve_entropy_weighted_avg(choice_entropy_map, baseline_sz)
    feature_info_gain = baseline_entropy - feature_entropy

    if feature_info_gain > max_info_gain:
      max_info_gain = feature_info_gain
      best_feat = feature

  return best_feat


In [ ]:
''' Corresponds with "Lecture 12: Decision trees" example with lecture data '''
# General arguments for all three tests
test_feature_choices, test_feature_index_map = retrieve_feature_choices_and_index_map(data_lecture, attribute_names_lecture)
test_attr_name_set = set(attribute_names_lecture)
test_target_types = list(retrieve_target_types(data_lecture))

test_subset_1 = data_lecture
test_baseline_sz_1 = 15
test_baseline_entropy_1 = 0.9968



test_best_feature_1 = retrive_best_feature(test_subset_1,
                                            test_feature_choices, 
                                            test_feature_index_map, 
                                            test_attr_name_set, 
                                            test_target_types, 
                                            test_baseline_sz_1, 
                                            test_baseline_entropy_1)

assert(test_best_feature_1 == "size")
test_attr_name_set.remove(test_best_feature_1)

test_subset_2 = retrieve_feature_choice_subset(test_subset_1, "large", 1)
test_baseline_sz_2 = 8
test_baseline_entropy_2 = 0.811


test_best_feature_2 = retrive_best_feature(test_subset_2,
                                           test_feature_choices, 
                                           test_feature_index_map, 
                                           test_attr_name_set, 
                                           test_target_types, 
                                           test_baseline_sz_2, 
                                           test_baseline_entropy_2)

assert(test_best_feature_2 == "color")
test_attr_name_set.remove(test_best_feature_2)

test_best_feature_3 = list(test_attr_name_set)[0]
assert(test_best_feature_3 == "shape")


<a id="train"></a>
## train

This function takes training_data, attribute names, and the depth limit and returns the decision tree as a nested dictionary. If the depth is 0, a dictionary is not returned. Instead, the mode of the target values is returned (i.e., majority class). 

* **training_data** List[List]: The data
* **attribute_names** List: The attribute names of the data (22 for mushroom; size, shape, and color for the lecture)
* **depth_limit** int: The depth limit of the tree


**returns** 
* **dt** Dict: The trained decision tree using the ID3 algorithm (entropy, information gain). It is represented as a nested dictionary. The dictionary returned for the lecture is structured as below:

```
{
('size', 1, 'large'): 
    {('color', 2, 'blue'): 'no', 
     ('color', 2, 'green'): 'yes', 
     ('color', 2, 'red'): 
         {('shape', 0, 'round'): 'yes', 
          ('shape', 0, 'square'): 'no'}
     }, 
('size', 1, 'small'): 
     {('shape', 0, 'square'): 'no', 
      ('shape', 0, 'round'): 
          {('color', 2, 'blue'): 'no', 
           ('color', 2, 'red'): 'yes', 
           ('color', 2, 'green'): 'no'}
      }
}
```


Notice that the keys are tuples; for instance, ('size', 1, 'large') is a key. The key includes the attribute's name, column number in data, and value.


The function currently returns a hard-coded tree. Your implementation should replace this with a tree that is learned from the data using the ID3 algorithm. You do not have to assert test `train`, but it may be worthwhile to check that it can return the tree from the lecture once your implementation is in place.

In [ ]:
def train(training_data, attribute_names, depth_limit=None):
  feature_choices, feature_index_map = retrieve_feature_choices_and_index_map(training_data, attribute_names)
  target_types_set = retrieve_target_types(training_data)
  attr_names_set, tar_types_list = set(attribute_names), list(target_types_set)
  default_label = get_mode(get_answers(training_data))

  def id3(subset, feature_choices, feature_index_map, remaining_feats, target_types, default, depth_limit):  
    if len(subset) == 0: return default # Base case: no more records to build tree from

    baseline_sz = len(subset)
    target_sz_1, target_sz_2 = retrieve_target_sizes(subset, target_types)
    baseline_entropy = entropy_hx(target_sz_1, target_sz_2, baseline_sz)

    if baseline_entropy == 0: return subset[0][-1]  # Base case: subset is homogenous 

    default_label = get_mode(get_answers(subset))
    if len(remaining_feats) == 0: return default_label # Base case: no more other features to split tree from

    if depth_limit and depth_limit == 0: return default_label
    if depth_limit and depth_limit > 0: depth_limit -= 1

    # Determine best feature to follow up from previous subset constrained to prior feature choice
    best_feat = retrive_best_feature(subset, feature_choices, feature_index_map, remaining_feats, target_types, baseline_sz, baseline_entropy)

    # This is where we recursively the tree, branching out from subtree to best feature
    next_node = {}
    next_best_feat_attr = remaining_feats.copy()
    next_best_feat_attr.remove(best_feat)

    for choice in feature_choices[best_feat]: # Go through the choices for next best feature, i.e domain
      subset_with_feat_choice = retrieve_feature_choice_subset(subset, choice, feature_index_map[best_feat]) 

      feat_branch = (best_feat, feature_index_map[best_feat], choice)
      child =  id3(subset=subset_with_feat_choice, 
                   feature_choices=feature_choices, 
                   feature_index_map=feature_index_map, 
                   remaining_feats=next_best_feat_attr, 
                   target_types=target_types, 
                   default=default_label,
                   depth_limit=depth_limit
                  )
      next_node[feat_branch] = child
    return next_node
  
  dt = id3(subset=training_data, 
           feature_choices=feature_choices, 
           feature_index_map=feature_index_map, 
           remaining_feats=attr_names_set, 
           target_types=tar_types_list, 
           default=default_label,
           depth_limit=depth_limit)
  
  return dt

In [ ]:
dt_lecture = train(training_data=train_lecture, attribute_names=attribute_names_lecture, depth_limit=0)

<a id="get_prediction"></a>
## get_prediction

This recursive function uses a decision tree represented as a nested dictionary get a prediction from a record, which is a row of the data. 

* **record** List[]: A row of data to be predicted
* **dt** the decision tree used to make the prediction


**returns** 
A prediction ('yes' or 'no' for instance, from our Self Check example.) 

In [ ]:
def get_prediction(record, dt):
    if not isinstance(dt, dict):
        return dt
    else:
        for key, value in dt.items():
            if record[key[1]]==key[2]:
                return get_prediction(record, value)

In [ ]:
print(get_prediction(['round','large','blue','no'], dt=dt_lecture))
print(get_prediction(['square','large','green','yes'], dt=dt_lecture))
print(get_prediction(['square','small','red','no'], dt=dt_lecture))

<a id="classify"></a>
## classify

This function takes a decision tree, observations, and a labeled flag to return a list of classifications. 

* **dt** Dict: The decision tree as a nested dictionary
* **observation** List[List]: a list of items, where each item is a row of the data
* **labeled** Bool: true for labeled data


**returns** 
* **y_hat** List: A list of classifications.

In [ ]:
def classify(dt, observations):
    y_hat = []
    for record in observations:
        y_hat.append(get_prediction(record, dt))   
    return y_hat

In [ ]:
print(classify(dt=dt_lecture, observations=test_lecture))

<a id="evaluate"></a>
## evaluate

This function evaluates the performance of a classifier. It takes a data set (training set or test set) and the classification result (see [classify](#classify) above and calculates the classification error rate:

$$error\_rate=\frac{errors}{n}$$ 

* **y_hat** List: A list of predictions
* **observations** List[List]: Data to be predicted (typically training or test set)


**returns** 

* **error_rate** float: The error rate.

In [ ]:
def evaluate(y_hat, observations):
    errors = 0
    ground_truth = get_answers(observations)
    for index in range(len(y_hat)):
        if y_hat[index] != ground_truth[index]:
            errors = errors + 1
    return errors / (len(y_hat))

In [ ]:
print(evaluate(classify(dt=dt_lecture, observations=data_lecture), observations=data_lecture))

<a id="get_stats"></a>
## get_stats

This function computes the mean and the standard deviation for a given list of observations. 

* **observations** List[float]: A list of observations


**returns** (mean, standard deviation) Tuple[float,float]: tuple consisting of mean and the standard deviation

In [ ]:
def get_stats(observations: List[float]) -> Tuple[float,float]:
    mean = sum(observations) / len(observations)
    variance = sum([(elem - mean)**2 for elem in observations]) / len(observations)
    std_dev = math.sqrt(variance)
    return mean, std_dev

In [ ]:
assert get_stats([2, 4, 4, 4, 5, 5, 7, 9]) == (5.0, 2.0)
assert get_stats([1, 1, 1]) == (1.0, 0.0)
assert get_stats([0]) == (0.0, 0.0)

<a id="cross_validate"></a>
## cross_validate

This function takes folds of data to `train`, `classify`, and `evaluate`.


* **folds** List[List[List]]: The original dataset partitioned into folds (see `create_folds` above)
* **attribute_names** int: the feature names
* **hyperparameters** List: A list of hyperparameters to explore (depth limits for a decision tree, for instance)

**returns** 

Nothing is returned, but for each hyperparameter setting, the function prints out the fold number and the error rate for that fold. The mean and variance is printed across folds for each hyperparameter setting. The error rates are reported in terms of percents.

In [ ]:
def cross_validate(folds, attribute_names, hyperparameters):
    for hyperparameter in hyperparameters:
        train_error, test_error  = [], []
        error_list_train, error_list_test = [], []
        for fold_index in range(len(folds)):
            training_data, test_data = create_train_test(folds, fold_index)
            tree = train(training_data=training_data, attribute_names=attribute_names, depth_limit=hyperparameter)
            y_hat_train = classify(tree, training_data)
            y_hat_test = classify(tree, test_data)
            error_rate_train = evaluate(y_hat_train, training_data)
            error_rate_test = evaluate(y_hat_test, test_data)
            error_list_train.append(error_rate_train)
            error_list_test.append(error_rate_test)
            print(f"Fold: {fold_index}\tTrain Error: {error_rate_train*100:.2f}%\tTest Error: {error_rate_test*100:.2f}%")
        print(f"***")
        print(f"Depth limit: {hyperparameter}")
        print(f"\nMean(Std. Dev.) over all folds:\n-------------------------------")
        print(f"Train Error: {get_stats(error_list_train)[0]*100:.2f}%({get_stats(error_list_train)[1]*100:.2f}%) Test Error: {get_stats(error_list_test)[0]*100:.2f}%({get_stats(error_list_test)[1]*100:.2f}%)")
        print("\n")

In [ ]:
cross_validate(folds=folds_lecture, attribute_names=attribute_names_lecture, hyperparameters=[0, 1, 2, 3, 4, 5, None])

<a id="pretty_print_tree"></a>
## pretty_print_tree

This function provides a text-based representation of a decision tree that is represented as a nested dictionary. 

* **dt** Dict: The decision tree as a nested dictionary
* **tab_space** Int: How much to tab successive depth levels of the resulting tree

In [ ]:
def pretty_print_tree(dt, tab_space):
    for key, value in dt.items():
        if isinstance(value, dict):
            print("  " * tab_space + str(key[0]).upper() + " - " + str(key[2]) + ": ")
            print("\n")
            pretty_print_tree(value, tab_space+3)
        else:
            print("  " * tab_space + str(key[0]).upper() + " - " + str(key[2]) + " =====> " + str(value))
            print("\n")

In [ ]:
dt_lecture = train(training_data=data_lecture, attribute_names=attribute_names_lecture, depth_limit=None)
pretty_print_tree(dt_lecture, tab_space=0)

<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <p>
        Let's work on the mushroom data. 
    </p>
</div>

## Classify the Mushrooom data

In [ ]:
folds_mushroom = create_folds(data=data_mushroom, n=10)

In [115]:
cross_validate(folds=folds_mushroom, attribute_names=attribute_names_mushroom, hyperparameters=[0, 1, 2, 3, 4, 5, None])

Fold: 4	Train Error: 0.00%	Test Error: 0.00%
Fold: 5	Train Error: 0.00%	Test Error: 0.00%
Fold: 6	Train Error: 0.00%	Test Error: 0.00%
Fold: 7	Train Error: 0.00%	Test Error: 0.00%
Fold: 8	Train Error: 0.00%	Test Error: 0.00%
Fold: 9	Train Error: 0.00%	Test Error: 0.00%
***
Depth limit: 4

Mean(Std. Dev.) over all folds:
-------------------------------
Train Error: 0.00%(0.00%) Test Error: 0.00%(0.00%)


Fold: 0	Train Error: 0.00%	Test Error: 0.00%
Fold: 1	Train Error: 0.00%	Test Error: 0.00%
Fold: 2	Train Error: 0.00%	Test Error: 0.00%


<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <p>
        Let's work on the mushroom data. 
    </p>
</div>

## Print the Mushroom Tree

In [ ]:
dt_mushroom = train(training_data=data_mushroom, attribute_names=attribute_names_mushroom, depth_limit=None)
pretty_print_tree(dt_mushroom, tab_space=0)

## Before You Submit...

1. Re-read the general instructions provided above, and
2. Hit "Kernel"->"Restart & Run All".